In [6]:
import pandas as pd
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import config
import random

random.seed(config.SEED)
from openai import OpenAI
client = OpenAI(api_key=config.OPENAI_API_KEY)

/scratch_local/mpb672-5192799/tmp/ipykernel_199723/3919594326.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [1]:
args = {"input_file": "cefr_leveled_texts.csv",
       "output_file": "controlled_texts_gpt4.csv"}
prompt_length = 50
num_stories = 10
level = "A1"

description = {
    "C2": "Can understand and interpret critically virtually all forms of the written language including abstract, structurally complex, or highly colloquial literary and non-literary writings. Can understand a wide range of long and complex texts, appreciating subtle distinctions of style and implicit as well as explicit meaning.",
    "C1": "Can understand in detail lengthy, complex texts, whether or not they relate to his/her own area of speciality, provided he/she can reread difficult sections.",
    "B2": "Can read with a large degree of independence, adapting style and speed of reading to different texts and purposes, and using appropriate reference sources selectively. Has a broad active reading vocabulary, but may experience some difficulty with low-frequency idioms.",
    "B1": "Can read straightforward factual texts on subjects related to his/her field and interest with a satisfactory level of comprehension.",
    "A2": "Can understand short, simple texts on familiar matters of a concrete type which consist of high frequency everyday or job-related language. Can understand short, simple texts containing the highest frequency vocabulary, including a proportion of shared international vocabulary items.",
    "A1": "Can understand very short, simple texts a single phrase at a time, picking up familiar names, words and basic phrases and rereading as required."
}

In [3]:
def write_story(level, story, max_len = 256, add_info=False):
    info = f", which is described as {description[level]}" if add_info else ""
    prompt = f"Continue the writing on CEFR level {level}{info}. Do not talk about the CEFR level. The story begins with "
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt + story}
    ]
    print(messages)
    response = client.chat.completions.create(model=config.OPENAI_MODEL, messages=messages, max_tokens=max_len)
    story = response.choices[0].message.content
    return story

In [7]:
cefr_texts = pd.read_csv('../dat/' + args['input_file'])
storyPrompts = cefr_texts.text.apply(lambda text: text[:text.find(' ', prompt_length)].strip().lstrip('\ufeff')).unique()
random.shuffle(storyPrompts)
if os.path.exists('../dat/' + args['output_file']):
    output_df = pd.read_csv('../dat/' + args['output_file'])
else:
    output_df = pd.DataFrame()
levels = ["A1", "A2", "B1", "B2", "C1", "C2"] if level is None else [level]

for story in storyPrompts[:num_stories]:
    print("_" * 100)
    print(story)
    for level in levels:
        print(level)
        if len(output_df) and len(output_df.loc[(output_df['label']==level) & (output_df['story']==story)]) > 0: continue
        text = write_story(level, story, add_info=False)
        print(text)
        new_row = {"label": level, "story": story, "text": text}
        pd.DataFrame([new_row]).to_csv('../dat/' + args['output_file'], mode='a', index=False, header=not os.path.exists('../dat/' + args.output_file))

____________________________________________________________________________________________________
I hate Professor Hernandez!
Me too! He keeps calling
A1
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Continue the writing on CEFR level A1. Do not talk about the CEFR level. The story begins with I hate Professor Hernandez!\nMe too! He keeps calling'}]
on me to answer questions in front of the whole class. It's so embarrassing, especially when I don't know the answer. And his homework assignments are the worst. They take forever to finish. But, I know we have to pass his class to move on to the next level.

Yesterday, he gave us this huge project to do, and we only have a week to finish it. I'm really worried about getting it done on time. Maybe we could work on it together? It might be easier if we help each other out. Plus, it could be more fun that way, instead of working alone.

What do you think? Should we meet at the library after sc

KeyboardInterrupt: 